In [4]:
import pandas as pd
import torch
import random
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.amp import autocast, GradScaler
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Custom Dataset class for ThaiSum
class ThaiSumDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_input_length=1024, max_target_length=256):
        self.texts = dataframe['body'].tolist()
        self.summaries = dataframe['summary'].tolist()
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        summary = str(self.summaries[idx])

        # Tokenize input (body)
        input_encoding = self.tokenizer(
            text,
            max_length=self.max_input_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Tokenize target (summary)
        target_encoding = self.tokenizer(
            summary,
            max_length=self.max_target_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encoding['input_ids'].squeeze(),
            'attention_mask': input_encoding['attention_mask'].squeeze(),
            'labels': target_encoding['input_ids'].squeeze()
        }

# q_t_given_0 function from LLADA
def q_t_given_0(input_ids, mask_token_id, t, N, tokenizer):
    """
    Remask tokens according to q_{t|0}: mask each token with probability s = t/N.
    """
    s = t / N
    special_tokens_mask = (input_ids == tokenizer.pad_token_id)  # Only protect pad tokens
    rand_mask = torch.bernoulli(torch.full(input_ids.shape, s)).bool().to(input_ids.device)
    mask_positions = rand_mask & ~special_tokens_mask

    masked_input = input_ids.clone()
    masked_input[mask_positions] = mask_token_id
    return masked_input, mask_positions

# Training step with LLADA diffusion and mixed precision
def training_step(model, tokenizer, batch, N, scaler):
    input_ids = batch['input_ids'].to(model.device)
    attention_mask = batch['attention_mask'].to(model.device)
    labels = batch['labels'].to(model.device)

    # Apply LLADA diffusion to input
    t = random.randint(1, N)
    masked_input_ids, _ = q_t_given_0(input_ids, tokenizer.pad_token_id, t, N, tokenizer)

    # Mixed precision forward pass
    with autocast(device_type='cuda'):
        outputs = model(
            input_ids=masked_input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss

    # Scale loss and backpropagate
    scaler.scale(loss).backward()
    return loss

# Training loop with mixed precision
def train_summarization_llada(model, tokenizer, dataset, optimizer, epochs=3, N=10, batch_size=8):
    model.train()
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    scaler = GradScaler()

    for epoch in range(epochs):
        total_loss = 0.0
        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            optimizer.zero_grad()
            loss = training_step(model, tokenizer, batch, N, scaler)
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloader)
        print(f"[Epoch {epoch+1}] Avg Loss: {avg_loss:.4f}")

def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Load dataset
    df = pd.read_csv('/kaggle/input/thaisum-train-10000-1024-nlpfinal/train-10000-1024.csv')

    # Initialize tokenizer and model
    model_name = 'google/mt5-small'  # ~300M parameters, supports Thai
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

    # Prepare dataset
    dataset = ThaiSumDataset(df, tokenizer, max_input_length=1024, max_target_length=256)

    # Set hyperparameters
    epochs = 1
    batch_size = 2
    N = 100  # For q_t_given_0
    learning_rate = 2e-5

    # Initialize optimizer
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    # Train the model
    train_summarization_llada(model, tokenizer, dataset, optimizer, epochs=epochs, N=N, batch_size=batch_size)

    # Save the model
    model.save_pretrained('./thai_summarization_llada_model_small')
    tokenizer.save_pretrained('./thai_summarization_llada_model_small')
    print("Model and tokenizer saved to './thai_summarization_llada_model_small'")

if __name__ == "__main__":
    main()

Using device: cuda


/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Epoch 1/1: 100%|██████████| 5000/5000 [18:06<00:00,  4.60it/s]


[Epoch 1] Avg Loss: nan
Model and tokenizer saved to './thai_summarization_llada_model_small'


In [5]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from torch.nn.functional import softmax

def infer_llada(input_text, model, tokenizer, L=128, N=10):
    # Move model to device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    # Tokenize input text
    input_encoding = tokenizer(
        input_text,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    ).to(device)

    # Initialize fully masked sequence of length L
    masked_ids = torch.full((1, L), tokenizer.pad_token_id, dtype=torch.long).to(device)
    r_t = masked_ids.clone()

    # Sampling steps
    for t in range(N, 0, -1):
        s = t / N

        # Predict next tokens
        with torch.no_grad():
            outputs = model(input_ids=input_encoding['input_ids'], decoder_input_ids=r_t)
            logits = outputs.logits  # Shape: (batch_size, seq_len, vocab_size)
            probs = softmax(logits, dim=-1)
            confidences, predicted_ids = torch.max(probs, dim=-1)

        # Update r_t with predicted tokens
        r_0 = r_t.clone()
        c = torch.ones_like(r_0, dtype=torch.float).to(device)  # Confidence scores

        for i in range(L):
            if r_t[0, i] != tokenizer.pad_token_id:  # If not masked
                r_0[0, i] = r_t[0, i]
                c[0, i] = 1.0
            else:
                r_0[0, i] = predicted_ids[0, i]
                c[0, i] = confidences[0, i]

        # Calculate number of unmasked tokens
        n_un = int(L * (1 - s))

        # Remask the n_un least confident positions
        if n_un > 0:
            _, lowest_conf_indices = torch.topk(c[0], n_un, largest=False)
            for idx in lowest_conf_indices:
                r_0[0, idx] = tokenizer.pad_token_id

        r_t = r_0.clone()

    # Final sequence
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_encoding['input_ids'],
            max_length=L,
            num_beams=1,
            early_stopping=True,
            decoder_start_token_id=tokenizer.pad_token_id
        )
    summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return summary

def main():
    # Load model and tokenizer
    model_path = './thai_summarization_llada_model_small'
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

    # Example input text
    input_text = "กีเก ซานเชซ ฟลอเรส\xa0 กุนซือเลือดกระทิงของทีมวัตฟอร์ด\xa0 เมินประเด็นจุดโทษปัญหาในเกมพรีเมียร์ลีก อังกฤษ นัดที่แตนอาละวาดเปิดบ้านพ่าย คริสตัล พาเลซ 0-1ชี้ทีมของเขาเล่นไม่ดีพอเอง,สำนักข่าวต่างประเทศรายงานวันที่ 27 ก.ย. ว่า กีเก ซานเชซ ฟลอเรส\xa0 ผู้จัดการทีมชาวสเปน ของ แตนอาละวาด วัตฟอร์ด\xa0 ยอมรับทีมของเขาเล่นได้ไม่ดีพอเอง ในเกมพรีเมียร์ลีก อังกฤษ นัดเปิดบ้านพ่าย อินทรีผงาด คริสตัล พาเลซ 0-1 เมื่อคืนวันอาทิตย์ที่ผ่านมา,เกมนี้จุดเปลี่ยนมาอยู่ที่การได้จุดโทษในช่วงครึ่งหลังของ คริสตัล พาเลซ ซึ่งไม่ค่อยชัดเจนเท่าไหร่ว่า อัลลัน นียอม นั้นไปทำฟาล์วใส่ วิลฟรีด ซาฮา ในเขตโทษหรือไม่ แต่ผู้ตัดสินก็ชี้เป็นจุดโทษ ซึ่ง โยอัน กาบาย สังหารไม่พลาด และเป็นประตูชัยช่วยให้ คริสตัล พาเลซ เอาชนะ วัตฟอร์ด ไป 1-0 และเป็นการพ่ายแพ้ในบ้านนัดแรกของวัตฟอร์ดในฤดูกาลนี้อีกด้วย,ฟลอเรส กล่าวว่า มันเป็นเรื่องยากในการหยุดเกมรุกของคริสตัล พาเลซ ซึ่งมันอึดอัดจริงๆสำหรับเรา เราเล่นกันได้ไม่ดีนักในตอนที่ได้ครองบอล เราต้องเล่นทางริมเส้นให้มากกว่านี้ เราไม่สามารถหยุดเกมสวนกลับของพวกเขาได้ และแนวรับของเราก็ยืนไม่เป็นระเบียบสักเท่าไหร่ในช่วงครึ่งแรก ส่วนเรื่องจุดโทษการตัดสินใจขั้นสุดท้ายมันอยู่ที่ผู้ตัดสิน ซึ่งมันเป็นการตัดสินใจที่สำคัญ ผมเองก็ไม่รู้ว่าเขาตัดสินถูกหรือเปล่า บางทีมันอาจเป็นจุดที่ตัดสินเกมนี้เลย แต่เราไม่ได้แพ้เกมนี้เพราะจุดโทษ เราแพ้ในวันนี้เพราะเราเล่นไม่ดีและคริสตัล พาเลซ เล่นดีกว่าเรา เราไม่ได้มีฟอร์มการเล่นที่ดีในเกมนี้เลย"  # Replace with actual input

    # Perform inference
    summary = infer_llada(input_text, model, tokenizer, L=256, N=10)
    print("Summary:", summary)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Summary: <extra_id_0> กล่าว
